In [71]:
import pandas as pd
import re

from sklearn.utils import shuffle
from transformers import BertTokenizerFast

PRE_TRAINED_MODEL = 'cimm-kzn/enrudr-bert'

Text normalization

In [72]:
emoji_dict = {' NEUTRAL_EMOJI ': '🤨🤫🤔😶🙈🙄😬🤷😑😐😵😲😗😮😳😯',
             ' POS_EMOJI ': """👌👍😂😀🤭🙃🥰❤️😘😍🖤😏♥💜🤣💕😅💚💙😉😆😁😊🌚😃😎🧡😋🤪🙂☻😄💖🤤☺🤠😇🌝🤩😜\
😌🧐💗🤗👩😹😸💛😝♡👏💓😛✌""",
             ' NEG_EMOJI ': '😕😫🤬🤦😱😰🥴😭💔😔😥👎🤒😩🤕😨🥵😢😖😡☹️😪🥺💀😓😣😞😿😠😟👿😒😦',
             ' HEALTH_EMOJI ': '🚑🤧💊💉😷🏥',
             '!': '¡❗️',
             '?': '¿❓',
             '!?': '⁉',
             '(': '（',
             '.': '。',
             'F': '🇫',
             'R': '🇷',
             'U': '🇺',
             'S': '🇸',
             '°C': '℃',
            }

In [73]:
punct = re.compile(r'[^\w@#,\.:;\\\|/\)\(\?!\-‑―—–\'"‘’«»„“”<>{}\[\]°%№&$₽€£®©™×~*+=\^\|`´]')

In [74]:
def normalize_tweet(tweet):
    tweet_norm = re.sub(r'(\xa0|\u200a|\u2028|\u3000|\u200f)', ' ', tweet)
    tweet_norm = re.sub(r'\xad', '', tweet_norm)
    tweet_norm = re.sub(r'&amp;', '&', tweet_norm)
    tweet_norm = re.sub(r'&gt;', '>', tweet_norm)
    tweet_norm = re.sub(r'&lt;', '<', tweet_norm)
    tweet_norm = re.sub(r'&quot;', '"', tweet_norm)
    tweet_norm = re.sub(r'&laquo;', '«', tweet_norm)
    tweet_norm = re.sub(r'&raquo;', '»', tweet_norm)
    tweet_norm = re.sub(r'&#8230;', '...', tweet_norm)
    tweet_norm = re.sub(r'&#13;', ' ', tweet_norm)
    tweet_norm = re.sub(r'…', '...', tweet_norm)
    tweet_norm = re.sub(r'Ё', 'Е', tweet_norm)
    tweet_norm = re.sub(r'ё', 'е', tweet_norm)
    tweet_norm = re.sub(r'@[A-z0-9]+', 'USERNAME', tweet_norm) # Twitter handle -> USERNAME
    tweet_norm = re.sub(r'https://t.co/[\w\d]+|HTTPURL_+|http[\./"\':]*', ' HTTPS ', tweet_norm) # URL -> HTTPS
    tweet_norm = re.sub(r'(pic)?\.twitter\.com/\w+', ' HTTPS ', tweet_norm) # URL -> HTTPS
    tweet_norm = re.sub(r'[0-9]+', 'NUM', tweet_norm) # number -> NUM
    for symb in tweet_norm: # some emojis -> NEUTRAL_EMOJI / POS_EMOJI / NEG_EMOJI / HEALTH_EMOJI
        for key in emoji_dict:
            if symb in emoji_dict[key]:
                tweet_norm = tweet_norm.replace(symb, key)
    tweet_norm = re.sub(punct, ' ', tweet_norm) # everything except alphanumeric symbols and punctuation marks -> space
    return tweet_norm.strip()

Preprocessing for NER task

In [75]:
tokenizer = BertTokenizerFast.from_pretrained(PRE_TRAINED_MODEL, do_lower_case=False)

In [76]:
def encode_labels(offsets, start_end_data):
    encoded_labels = ['O'] * len(offsets)
    for i in range(len(start_end_data)):
        s = start_end_data[i]['start']
        e = start_end_data[i]['end']
        for idx, mapping in enumerate(offsets):
            if mapping[0] == s:
                encoded_labels[idx] = 'B'
            elif mapping[0] > s and mapping[1] <= e:
                encoded_labels[idx] = 'I'
    return encoded_labels

In [77]:
def tokenize_text(tokens, subword_labels):
    new_tokens = []
    labels = []
    for i, token in enumerate(tokens):
        if not token.startswith('##'):
            new_tokens.append(token)
            labels.append(subword_labels[i])
        else:
            new_tokens[-1] += token[2:]
    return new_tokens, labels

In [78]:
def tok_and_label(sent, ent, tokenizer):
    encoding = tokenizer.encode_plus(sent,
                                     return_offsets_mapping=True,
                                     return_token_type_ids=False,
                                     return_attention_mask=False,
                                     add_special_tokens=False,
                                    )
    tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
    offsets = encoding['offset_mapping']
    subword_labels = encode_labels(offsets, ent)
    words, labels = tokenize_text(tokens, subword_labels)
    assert len(words) == len(labels)
    return words, labels

In [79]:
def dump_labeled_texts(df, path, tokenizer):
    with open(path, 'w', encoding='utf-8') as f:
        f.write('-DOCSTART\n')
        texts = df['text'].tolist()
        ents = df['ent'].tolist()
        for i in range(len(texts)):
            words, labels = tok_and_label(texts[i], ents[i], tokenizer)
            for j in range(len(words)):
                f.write(f'{words[j]} {labels[j]}\n')
            f.write('\n')

New span identification for normalized texts (NER task)

In [80]:
def find_limits(text, span, lang='rus'):
    start = -1
    end = -1
    for i in range(len(text)):
        if lang == 'eng':
            new_text = text.lower()
            new_span = span.lower()
        elif lang == 'rus':
            new_text = text
            new_span = span
        if new_text[i] == new_span[0]:
            if new_text[i:i+len(new_span)] == new_span:
                start = i
                end = i + len(new_span)
    return start, end

#### Normalization for classification task

Input format: .tsv (tweet_id<font color='blue'>\t</font>tweet<font color='blue'>\t</font>label<font color='blue'>\n</font>)

Output format: .tsv (tweet_id<font color='blue'>\t</font>normalized tweet<font color='blue'>\t</font>label<font color='blue'>\n</font>)

In [81]:
train = pd.read_csv('orig_train.tsv', sep='\t', index_col='tweet_id')

In [25]:
train['tweet'] = train['tweet'].apply(normalize_tweet)

In [27]:
train.to_csv('train.tsv', sep='\t')

#### Preprocessing for NER task without normalization

Input format: .tsv (tweet_id<font color='blue'>\t</font>text<font color='blue'>\t</font>start<font color='blue'>\t</font>end<font color='blue'>\t</font>span<font color='blue'>\n</font>)

Output format: .tsv (BIO scheme)<br>
-DOCSTART<br>
word1 B<br>
word2 I<br>
...<br>
wordN O<br>

word1 O<br>
...

In [82]:
orig_train = pd.read_csv('orig_train.tsv', sep='\t', index_col=['tweet_id', 'text'])
orig_train['ent'] = orig_train.apply(lambda x: {'start': x['start'], 'end': x['end'], 'span': x['span']}, axis=1)
orig_train.drop(['start', 'end', 'span'], axis=1, inplace=True)
train = orig_train.groupby(['tweet_id', 'text']).agg(list).reset_index().set_index('tweet_id')

In [85]:
dump_labeled_texts(train, 'train.tsv', tokenizer)

#### Preprocessing and normalization for NER task

In [111]:
orig_train = pd.read_csv('orig_train.tsv', sep='\t', index_col='tweet_id')

In [112]:
orig_train['text'] = orig_train['text'].apply(normalize_tweet)
orig_train['span'] = orig_train['span'].apply(normalize_tweet)

In [113]:
orig_train['start'] = orig_train.apply(lambda x: find_limits(x['text'], x['span'], lang='rus')[0], axis=1)
orig_train['end'] = orig_train.apply(lambda x: find_limits(x['text'], x['span'], lang='rus')[1], axis=1)

In [114]:
orig_train['ent'] = orig_train.apply(lambda x: {'start': x['start'], 'end': x['end'], 'span': x['span']}, axis=1)
orig_train.drop(['start', 'end', 'span'], axis=1, inplace=True)
train = orig_train.groupby(['tweet_id', 'text']).agg(list).reset_index().set_index('tweet_id')

In [116]:
dump_labeled_texts(train, 'train.tsv', tokenizer)